在实际工作中获取到的数据往往不那么理想，**可能会存在非数值类型的文本数据、重复值、缺失值、异常值及数据分布不均衡**等问题，因此，在进行数学建模前还需要对这些问题进行处理，这项工作称为特征工程。特征工程通常分为**特征使用方案、特征获取方案、特征处理、特征监控**几大部分，其中特征处理是特征工程的核心内容，有时称为数据预处理。

# 非数值类型数据处理
## 一、Get_dummies哑变量处理
哑变量也叫虚拟变量，通常取值为0或1，比如将性别中的“男”和“女”分别转换成数字1和0就是哑变量最经典的应用。

在Python中，通常利用pandas库中的get_dummies()函数进行哑变量处理，它不仅可以处理“男”和“女”这种只有两个分类的简单问题，还可以处理含有多个分类的问题。

### 1、简单示例：“男”和“女”的数值转换

In [1]:
import pandas as pd
df = pd.DataFrame({'客户编号': [1,2,3], '性别': ['男', '女', '男']})
df.head()

,客户编号,性别
0,1,男
1,2,女
2,3,男


In [2]:
df = pd.get_dummies(df, columns=['性别']) # get_dummies()函数构造哑变量，get_dummies()函数的第1个参数为表格名称，第2个参数为需要处理的列的名称
df.head()

,客户编号,性别_女,性别_男
0,1,0,1
1,2,1,0
2,3,0,1


In [3]:
# 去除多重共线性 （性别_男＝1-性别_女）
df = df.drop(columns='性别_女')
df = df.rename(columns={'性别_男':'性别'})
df.head()

,客户编号,性别
0,1,1
1,2,0
2,3,1


#### 复杂示例：房屋朝向的数值转换

In [4]:
import pandas as pd
df = pd.DataFrame({'房屋编号': [1, 2, 3, 4, 5], '朝向': ['东','南', '西', '北', '南']})
df.head()

,房屋编号,朝向
0,1,东
1,2,南
2,3,西
3,4,北
4,5,南


In [5]:
df = pd.get_dummies(df, columns=['朝向']) # get_dummies()函数构造哑变量，get_dummies()函数的第1个参数为表格名称，第2个参数为需要处理的列的名称
df.head()

,房屋编号,朝向_东,朝向_北,朝向_南,朝向_西
0,1,1,0,0,0
1,2,0,0,1,0
2,3,0,0,0,1
3,4,0,1,0,0
4,5,0,0,1,0


上表同样存在多重共线性（即根据3个朝向的数字就能判断第4个朝向的数字是0还是1），因此需要从新构造出来的4个哑变量中删去1个，假设删去“朝向_西”列

In [6]:
df = df.drop(columns='朝向_西')
df.head()

,房屋编号,朝向_东,朝向_北,朝向_南
0,1,1,0,0
1,2,0,0,1
2,3,0,0,0
3,4,0,1,0
4,5,0,0,1


构造哑变量容易产生高维数据，因此，哑变量常和PCA（主成分分析）一起使用，即构造哑变量产生高维数据后采用PCA进行降维。

## 二、Label Encoding编号处理
除了使用get_dummies()函数进行非数值类型数据处理外，还可以使用Label Encoding进行编号处理，具体来说，是使用LabelEncoder()函数将文本类型的数据转换成数字。

In [8]:
df = pd.DataFrame({'编号': [1, 2, 3, 4, 5], '城市': ['北京', '上海', '广州', '深圳', '北京']})
df.head()

,编号,城市
0,1,北京
1,2,上海
2,3,广州
3,4,深圳
4,5,北京


In [9]:
# 将“城市”列的文本内容转换为不同的数字
from sklearn.preprocessing import LabelEncoder # 从Scikit-Learn库的preprocessing模块中引入LabelEncoder()函数
le = LabelEncoder() # 将LabelEncoder()函数赋给变量le
label = le.fit_transform(df['城市']) # 用fit_transform()函数将待转化的列传入模型中进行拟合，并将结果赋给变量label
label

array([1, 0, 2, 3, 1])

In [10]:
df['城市'] = label
df.head()

,编号,城市
0,1,1
1,2,0
2,3,2
3,4,3
4,5,1


上述示例中使用Label Encoding处理后产生了一个奇怪的现象：上海和广州的平均值是北京，这个现象其实是没有现实意义的，这也是Label Encoding的一个缺点——可能会产生一些没有意义的关系。不过树模型（如决策树、随机森林及XGBoost等集成算法）能很好地处理这种转化，因此对于树模型来说，这种奇怪的现象是不会影响结果的。

#### 补充知识点：pandas库中的replace()函数
LabelEncoder()函数生成的数字是随机的，如果想按特定内容进行替换，可以使用replace()函数。

In [11]:
df = pd.DataFrame({'编号': [1, 2, 3, 4, 5], '城市': ['北京', '上海', '广州', '深圳', '北京']})
df.head()

,编号,城市
0,1,北京
1,2,上海
2,3,广州
3,4,深圳
4,5,北京


在使用replace()函数之前，先利用value_counts()函数查看“城市”列有哪些内容需要替换（因为有时数据量很大，通过人眼判断可能会遗漏某些内容）

In [12]:
df['城市'].value_counts()

北京    2
上海    1
广州    1
深圳    1
Name: 城市, dtype: int64

In [13]:
df['城市'] = df['城市'].replace({'北京': 0, '上海': 1, '广州': 2, '深圳':3})
df.head()

,编号,城市
0,1,0
1,2,1
2,3,2
3,4,3
4,5,0


## 总结
Get_dummies的优点是它的值只有0和1，缺点是当类别的数量很多时，特征维度会很高，此时可以配合使用下一章要讲解的PCA（主成分分析）来减少维度。如果类别数量不多，可以优先考虑使用Get_dummies，其次考虑使用Label Encoding或replace()函数；但如果是基于树模型的机器学习模型，用Label Encoding也没有太大关系。